# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
MAX_TURN = 3

## The task!

In [3]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, the hub of cutting-edge deep learning technology. With courses authored by industry expert Andrew Ng, this platform offers a gateway to understanding and applying deep learning principles. Whether you're a novice or an expert, DeepLearning.AI provides a seamless learning experience for all levels. From neural networks to natural language processing, the possibilities are endless. Join the community, elevate your AI skills, and explore the endless opportunities that await in the realm of deep learning. Spark your curiosity and transform your future with DeepLearning.AI!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=MAX_TURN,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, the hub of cutting-edge deep learning technology. With courses authored by industry expert Andrew Ng, this platform offers a gateway to understanding and applying deep learning principles. Whether you're a novice or an expert, DeepLearning.AI provides a seamless learning experience for all levels. From neural networks to natural language processing, the possibilities are endless. Join the community, elevate your AI skills, and explore the endless opportunities that await in the realm of deep learning. Spark your curiosity and transform your future with DeepLearning.AI!

-------------------------

## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": MAX_TURN},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": MAX_TURN},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": MAX_TURN},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": MAX_TURN},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=MAX_TURN,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, the hub of cutting-edge deep learning technology. With courses authored by industry expert Andrew Ng, this platform offers a gateway to understanding and applying deep learning principles. Whether you're a novice or an expert, DeepLearning.AI provides a seamless learning experience for all levels. From neural networks to natural language processing, the possibilities are endless. Join the community, elevate your AI skills, and explore the endless opportunities that await in the realm of deep learning. Spark your curiosity and transform your future with DeepLearning.AI!

-------------------------


--------------------------------------------------------------------------------
Legal Reviewer (to Critic):

Legal Reviewer: 

1. Verify trademark infringement risk for "DeepLearning.AI" and "Andrew Ng."
2. Substantiate claims regarding platform capabilities to avoid false advertising.
3. Consider adding a disclaimer to manage user expectations and mitigate legal disputes.

--------------------------------------------------------------------------------
Critic (to Legal Reviewer):

Legal Review:

1. Trademark Infringement Risk: Conduct a thorough investigation to assess whether the use of "DeepLearning.AI" and "Andrew Ng" poses a risk of trademark infringement. Verify the ownership and validity of these trademarks to ensure compliance with intellectual property laws.

2. Substantiation of Claims: Ensure that all claims made about the platform's capabilities are accurate and supported by evidence. Avoid false advertising by substantiating these statements with reliable data or example


--------------------------------------------------------------------------------
Ethics Reviewer (to Critic):

To further enhance the ethical integrity of the content promoting DeepLearning.AI, consider the following suggestions:

4. User Consent: Clearly explain how user data is collected and provide options for users to manage their preferences, emphasizing the importance of obtaining user consent for data processing.

5. Security Measures: Highlight the security measures in place to protect user data from unauthorized access or breaches, reassuring users of the platform's commitment to data security.

6. Feedback Mechanisms: Establish channels for users to provide feedback, report concerns, or address ethical issues, fostering a responsive and transparent environment for users.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

******************

Overall, the feedback from the SEO and Legal reviewers provides comprehensive guidance for enhancing the content. By addressing SEO recommendations such as keyword incorporation, meta tag optimization, and clear calls-to-action, you can improve search engine visibility and user engagement. Similarly, implementing legal suggestions like conducting trademark infringement checks, substantiating platform claims, and adding disclaimers will ensure compliance and transparency.

Balancing SEO and legal considerations is essential for creating trustworthy and high-quality content. By following these suggestions, you can create a well-rounded piece that meets SEO standards and legal requirements while minimizing potential risks. It is recommended to carefully integrate these recommendations to create content that is both optimized for search engines and legally sound, ultimately enhancing the overall quality and credibility of the content.

------------------------------------------------------


--------------------------------------------------------------------------------
Legal Reviewer (to Critic):

As a Legal Reviewer:

1. Ensure that any advice provided in the content regarding trademark checks, claim verification, and disclaimers aligns accurately with applicable legal requirements to avoid any misleading information.
   
2. Verify that the recommendations on SEO practices do not compromise legal obligations, such as accuracy in information or promotional regulations, to prevent potential legal risks.
   
3. Confirm that the content strikes a balance between SEO strategies and legal compliance, emphasizing the importance of upholding legal integrity throughout all optimization efforts.

--------------------------------------------------------------------------------
Critic (to Legal Reviewer):

As a Legal Reviewer, here is feedback on the content provided:

1. Advice on trademark checks, claim verification, and disclaimers should be accurate and aligned with applicable


--------------------------------------------------------------------------------
Ethics Reviewer (to Critic):

Thank you for your positive feedback. If you have any specific concerns about elements of the content that need further attention in terms of accuracy, credibility, or ethical standards, please let me know. I am here to provide additional detailed guidance as needed.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Meta Reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 
{'Reviewer': 'SEO Specialist', 'Review': 'Thank you for the positive feedback, and I'm glad you found the suggestions helpful. Remember, focusing on readability, engagement, and educational value will ultimately lead to improved q


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Based on the aggregated feedback from all reviewers, it is clear that the content has the potential for improvement in various aspects such as readability, engagement, SEO strategies, legal compliance, accuracy, credibility, and ethical standards. The reviewers have provided valuable insights into ways to enhance the content to better resonate with the audience while aligning with legal and ethical considerations.

Moving forward, it is essential to focus on optimizing the content for both search engines and readers to improve rankings and user satisfaction. Integrating legal requirements, maintaining accuracy, and upholding ethical standards will help build trust with the audience and ensure the content's integrity.

By incorporating SEO strategies effectively, enhancing readability, and maximizing educational value, the content can achieve better visibility and engagement. Co

## Get the summary

In [16]:
print(res.summary)

Title: "Elevating Your Content: A Comprehensive Guide to Enhancing Readability, SEO, and Legal Compliance"

Ensuring your content resonates with readers and search engines alike is vital for success. By heeding feedback on readability, engagement, SEO, legal compliance, accuracy, credibility, and ethical standards, you pave the way for improvement. Focus on optimizing for both algorithms and audience satisfaction to boost visibility and trust. Integrate SEO strategies, enhance readability, and uphold legal and ethical standards to create high-quality, impactful content. Implementing reviewer suggestions will refine your content strategy, leading to increased engagement and relevance. Embrace continuous improvement in these critical areas to craft compelling, compliant content that captivates your audience effectively.
